In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
import pandas as pd

In [309]:
res = torch.load('results\epoch95.t7')

In [310]:
weights = res['updates']
peers_types = res['peers_types']

In [311]:
weights[0]['linear.weight']

tensor([[ 0.0748,  0.1628,  0.0902,  ...,  0.0602, -0.0554,  0.0182],
        [-0.0898, -0.1014, -0.0681,  ..., -0.0515, -0.1129, -0.0899],
        [ 0.0680,  0.0061, -0.0075,  ..., -0.0129,  0.0588,  0.1297],
        ...,
        [ 0.1532, -0.0812, -0.0807,  ..., -0.0724,  0.0637, -0.0352],
        [-0.0870,  0.1223,  0.0606,  ...,  0.1845,  0.0897,  0.0946],
        [-0.0932,  0.1241, -0.0756,  ..., -0.0768,  0.0004, -0.0579]],
       device='cuda:0')

In [312]:
n = len(peers_types)

In [313]:
weights_list = [[] for i in range(n)]
last_layer_list = [[] for i in range(n)]

In [314]:
layers = weights[0].keys()
for i in range(n):
    for layer in layers:
        x = weights[i][layer].squeeze().flatten().cpu().numpy()
#         weights_list[i]= x if not len(weights_list[i]) else np.concatenate((weights_list[i], x))
        if layer == 'linear.weight':
            last_layer_list[i]= x 

In [315]:
updates_dists = np.zeros([n, n])
last_layer_dists = np.zeros([n, n])
for i in range(n):
    updates_dists[i] = distance.cdist([weights_list[i]], weights_list)
    last_layer_dists[i] = distance.cdist([last_layer_list[i]], last_layer_list)

In [316]:
updates_dists = np.sort(updates_dists, axis = 1)
last_layer_dists = np.sort(last_layer_dists, axis = 1)

In [317]:
updates_dists = updates_dists.T
updates_dists = updates_dists[:-8]
last_layer_dists = last_layer_dists.T
last_layer_dists = last_layer_dists[:-8]

In [318]:
updates_dists = updates_dists.T
last_layer_dists = last_layer_dists.T

In [319]:
dists = np.sum(last_layer_dists, axis=1)

In [320]:
indices = dists.argsort()

In [321]:
good_idx = indices[:-8]
bad_idx = indices[-8:]

In [331]:
g_u = []
b_u = []
for idx in indices:
    if idx in good_idx:
        g_u.append(last_layer_list[idx].reshape(10, 512))
    else:
        b_u.append(last_layer_list[idx].reshape(10, 512))

In [332]:
avg = g_u[0]
for j in range(1, len(g_u)):
    avg+= g_u[j]
avg/= len(g_u)

In [333]:
dist2 = []
for b in b_u:
    dist = []
    for i in range(10):
        d = sum(abs(b[i, :]-avg[i, :]))
        dist = d if not len(dist) else np.concatenate((dist, d))
    dist2.append(dist)

TypeError: object of type 'numpy.float64' has no len()

In [325]:
dist = []
for d in dist2:
    top3 = d.argsort()[-3:]
    dist = top3 if not len(dist) else np.concatenate((dist, top3))

In [326]:
a = set(dist)
freq = {}
for x in a:
    freq[x] = np.where(dist == x)[0].sum()

In [327]:
sorted(freq.items(), key=lambda item: item[1])

[(0, 0),
 (6, 2),
 (9, 10),
 (1, 18),
 (8, 24),
 (4, 28),
 (5, 43),
 (7, 43),
 (3, 52),
 (2, 56)]

In [ ]:
df1 = pd.DataFrame({'peer_type': peers_types, 'd1':np.sum(updates_dists, axis = 1)})
df2 = pd.DataFrame({'peer_type': peers_types, 'd2':np.sum(last_layer_dists, axis = 1)})

In [ ]:
df2[df2['peer_type'] == 'Bad update'].d2.mean()

In [ ]:
df2[df2['peer_type'] == 'Good update'].d2.mean()

In [ ]:
good_updates = []
bad_updates = []
good_last_layers = []
bad_last_layers = []
set(peers_types)

In [ ]:
for i, t in enumerate(peers_types):
    if t == 'Good update':
        good_updates= weights_list[i] if not len(good_updates) else np.concatenate((good_updates, weights_list[i]))
        good_last_layers= last_layer_list[i] if not len(good_last_layers) else np.concatenate((good_last_layers, last_layer_list[i]))
    else:
        bad_updates= weights_list[i] if not len(bad_updates) else np.concatenate((bad_updates, weights_list[i]))
        bad_last_layers= last_layer_list[i] if not len(bad_last_layers) else np.concatenate((bad_last_layers, last_layer_list[i]))

In [ ]:

fig,ax = plt.subplots(1)

ax.hist(good_updates, alpha=0.4, bins= 10, label='Good Updates')
ax.hist(bad_updates, alpha=0.6, bins= 10, label='Bad Updates', color = 'red')
plt.legend(loc='upper right')
ax.set_yticks([])
ax.set_xticks([])
# plt.savefig('cifar10_resnet18_dist_all.png',  bbox_inches='tight', dpi = 600)
plt.show()

In [ ]:
fig,ax = plt.subplots(1)
ax.hist(good_last_layers, alpha=0.4, bins = 300, label='Good Updates')
ax.hist(bad_last_layers, alpha=0.6, bins = 300, label='Bad Updates', color = 'red')
plt.legend(loc='upper right')
ax.set_yticks([])
ax.set_xticks([])
# plt.savefig('cifar10_resnet18_dist_last.png',  bbox_inches='tight', dpi = 600)
plt.show()